In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
url1 = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_train.csv'
df = pd.read_csv(url1)

In [6]:
df.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work,target_reg,target_class
0,y13744087j,Kia Rio X-line,economy,petrol,3.78,2015,76163,2021,9492.96,another_bug
1,O41613818T,VW Polo VI,economy,petrol,3.90,2015,78218,2021,2656.23,electro_bug
2,d-2109686j,Renault Sandero,standart,petrol,6.30,2012,23340,2017,1526.11,gear_stick
3,u29695600e,Mercedes-Benz GLC,business,petrol,4.04,2011,1263,2020,1338.00,engine_fuel
4,N-8915870N,Renault Sandero,standart,petrol,4.70,2012,26428,2017,825.72,engine_fuel


# Start with basic dataframe to get Baseline

Make a function preparing dataset

In [57]:
def prep(df):
    if df is None:
        return 0
    
    temp = df.copy()
    temp = temp[['model', 'car_type', 'fuel_type', 'car_rating', 'year_to_start', 'riders', 'year_to_work']]
    
    # make ohe for columns car_type and fuel_type
    ohe = pd.get_dummies(df[['car_type', 'fuel_type']]).drop(['car_type_business', 'fuel_type_electro'], axis=1)
    temp = pd.concat([temp, ohe], axis=1).drop(['car_type', 'fuel_type'], axis=1)
    
    
    return temp

In [12]:
from sklearn.model_selection import train_test_split

Make standart Train Test Split

In [63]:
X = prep(df)
y = df['target_class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

Lets take default catboost

In [60]:
from catboost import CatBoostClassifier

In [66]:
cat_cols = ['model', 'year_to_start', 'year_to_work']
ctb = CatBoostClassifier(iterations=300).fit(X_train, y_train, 
                                             eval_set=(X_test, y_test), 
                                             cat_features=cat_cols, 
                                             early_stopping_rounds=20,                                         
                                             verbose=20)

Learning rate set to 0.173181
0:	learn: 2.1934731	test: 2.1958543	best: 2.1958543 (0)	total: 15ms	remaining: 4.5s
20:	learn: 2.0692537	test: 2.1918308	best: 2.1894681 (14)	total: 964ms	remaining: 12.8s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 2.189468085
bestIteration = 14

Shrink model to first 15 iterations.


# Lets Validate

In [69]:
url2 = 'https://raw.githubusercontent.com/a-milenkin/Competitive_Data_Science/main/data/car_test.csv'
df_val = pd.read_csv(url2)

In [71]:
df_val.head()

,car_id,model,car_type,fuel_type,car_rating,year_to_start,riders,year_to_work
0,P17494612l,Skoda Rapid,economy,petrol,4.80,2013,42269,2019
1,N-1530212S,Renault Sandero,standart,petrol,4.32,2015,90014,2016
2,B-1154399t,Smart ForTwo,economy,petrol,4.46,2015,82684,2017
3,F12725233R,Smart ForFour,economy,petrol,2.80,2014,68833,2021
4,l-1139189J,Skoda Rapid,economy,petrol,6.56,2013,42442,2021


In [73]:
df_val['target_class'] = ctb.predict(prep(df_val))

result = df_val[['car_id','target_class']]

In [75]:
result.to_csv('answer.csv', index=False)